# Lesson 5: Tools And Routing

In [ ]:
import os
import openai
import datetime
import requests
import wikipedia
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec
from pydantic import BaseModel, Field
from langchain.agents import tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

from dotenv import load_dotenv

load_dotenv()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
@tool
def search(query: str) -> str:
    """search for weather online"""
    return "42f"


search.name

In [ ]:
search.description

In [ ]:
search.args

In [ ]:
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [ ]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [ ]:
search.args

In [ ]:
search.run("sf")

In [ ]:
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")


@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.now(datetime.timezone.utc)
    time_list = [datetime.datetime.fromisoformat(time_str).replace(tzinfo=datetime.timezone.utc) for time_str in
                 results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'

In [ ]:
get_current_temperature.name

In [ ]:
get_current_temperature.description

In [ ]:
get_current_temperature.args

In [ ]:
from langchain.tools.render import format_tool_to_openai_function

format_tool_to_openai_function(get_current_temperature)

get_current_temperature({"latitude": 13, "longitude": 14})

In [ ]:
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
                wikipedia.exceptions.PageError,
                wikipedia.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [ ]:
search_wikipedia.name

In [ ]:
search_wikipedia.description

In [ ]:
format_tool_to_openai_function(search_wikipedia)

In [ ]:
search_wikipedia({"query": "langchain"})

# Routing

In [ ]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [ ]:
model.invoke("what is langchain")

In [ ]:
model.invoke("what is rge weather in sf right now")

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [ ]:
chain.invoke({"input": "what is the weather in sf right now"})


In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
result = chain.invoke(({"input": "what is the weather in sf right now"}))

type(result)
result.tool
result.tool_input

In [ ]:
get_current_temperature(result.tool_input)
result = chain.invoke({"input": "hi"})

In [ ]:
type(result)
result.return_values

In [ ]:
from langchain.schema.agent import AgentFinish


def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [ ]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})
result

In [ ]:
result = chain.invoke({"input": "What is langchain?"})
result

In [ ]:
chain.invoke({"input": "hi"})